In [ ]:
import pandas as pd
import os
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import json
from IPython.display import display, Markdown

import sys
sys.path.append('../')

from evalutils.roc import get_bootstrapped_roc_ci_curves
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from utilities import data, roc, threshold, output, info
from utilities.info import *

In [ ]:
# FIG_DIR = f"{TEAMS_DIR}/melba-figs"
# TAB_DIR = f"{TEAMS_DIR}/melba-tables"

print("FILES:\t", FILE_DIR)
print("RESULT:\t", RESULTS_DIR)
print("FIGS:\t", FIG_DIR)
print("TABLES:\t", TAB_DIR)

In [ ]:
POLICIES_TO_USE = ['90% Sensitivity', '90% Specificity', 'Brock ILST (6%)']
TABLE_SCORE_PRECISION = 2
MIN_MAL = 15
TOP_N_CONFOUNDERS = 10

In [ ]:
nlst_preds_nodule = pd.read_csv(f"{FILE_DIR}/nlst_allmodels_demos.csv")
nlst_preds_nodule.info()

In [ ]:
with open(f'{FILE_DIR}/nlst_democols.json') as json_data:
    nlst_democols_og = json.load(json_data)
    json_data.close()

nlst_democols_og

# Materials

## Demographic characteristics of the NLST validation set (n=5911 scans).

In [ ]:
nlst_5911, nlst_democols, _ = data.prep_nlst_preds(nlst_preds_nodule, nlst_democols_og, scanlevel=True, sybil=True, pretty=True, bin_num=False)
len(nlst_5911)

In [ ]:
df_out, invalid_attributes_5911 = output.dataset_malben_table(nlst_5911, nlst_democols)
df_out

In [ ]:
demos_no_duplicates = [data.rename_cols[c] for c in ['race','ethnic','educat','Gender','weight','height', 'BMI', 'Age']]
df_out2 = df_out.xs('Demographics', level='Category').query("Characteristic in @demos_no_duplicates")
df_out2

In [ ]:
print(df_out2.style.format(precision=1, na_rep='-').format_index(output.latex_replace_arrowbrackets).format(output.latex_replace_arrowbrackets).to_latex(
    buf=f"{TAB_DIR}/datasetDemos.tex",
    hrules=True, label='tab:datasetDemos', environment='table*', column_format='ll|rrr', position='ht!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data',
    caption=f'Demographic characteristics of the NLST validation set (n={len(nlst_5911)} scans). HS = High School.'))

### Venk21 Training Set

In [ ]:
venk21_full, venk21_democols, _ = data.prep_nlst_preds(nlst_preds_nodule, nlst_democols_og, scanlevel=True, sybil=False, pretty=True, bin_num=False)
len(venk21_full)

In [ ]:
venk21_datatable, _ = output.dataset_malben_table(venk21_full, venk21_democols)
venk21_datatable = venk21_datatable.xs('Demographics', level='Category').query("Characteristic in @demos_no_duplicates")
venk21_datatable

In [ ]:
print(venk21_datatable.style.format(precision=1, na_rep='-').format_index(output.latex_replace_arrowbrackets).format(output.latex_replace_arrowbrackets).to_latex(
    buf=f"{TAB_DIR}/venk21trainData.tex",
    hrules=True, label='tab:venk21trainData', environment='table*', column_format='ll|rrr', position='ht!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data',
    caption=f'Demographics of the NLST training set used for Venkadesh21 (n={len(venk21_full)} scans). HS = High School.'))

### Sybil Training Set

In [ ]:
sybil_train = pd.read_csv(f"{FILE_DIR}/nlst_sybil_demos.csv").query('split == "train"')
sybil_train['label'] = sybil_train['sybil_label']
sybil_train.info()

In [ ]:
with open(f'{FILE_DIR}/nlst_sybil_democols.json') as json_data:
    sybil_democols = json.load(json_data)
    json_data.close()

In [ ]:
sybil_train, sybil_democols = data.nlst_pretty_labels(sybil_train, sybil_democols)
sybil_democols

In [ ]:
sybil_datatable, _ = output.dataset_malben_table(sybil_train, sybil_democols)
sybil_datatable = sybil_datatable.xs('Demographics', level='Category').query("Characteristic in @demos_no_duplicates")
sybil_datatable

In [ ]:
print(sybil_datatable.style.format(precision=1, na_rep='-').format_index(output.latex_replace_arrowbrackets).format(output.latex_replace_arrowbrackets).to_latex(
    buf=f"{TAB_DIR}/sybilTrainData.tex",
    hrules=True, label='tab:sybilTrainData', environment='table*', column_format='ll|rrr', position='ht!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data',
    caption=f'Demographics of the NLST training set used for Sybil (n={len(sybil_train)} scans). HS = High School.'))

# Methods

### Thresholds from policies on validation sets.

In [ ]:
nlst_5911, _, _ = data.prep_nlst_preds(nlst_preds_nodule, nlst_democols_og, scanlevel=True, sybil=True, pretty=True, bin_num=False)
nlst_5911_policies, _ = threshold.get_threshold_policies(nlst_5911, models=output.NLST_5911_MODELCOLS, policies=[("Sensitivity", 0.9),("Specificity", 0.9)], brock=False)

In [ ]:
nlst_5911_policies.rename(columns=output.RENAME_POLICIES, inplace=True)
nlst_5911_policies

In [ ]:
print(nlst_5911_policies.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/thresholds.tex",
    hrules=True, label='tab:thresholds', environment='table', column_format='l|rr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data',
    caption='Thresholds used to evaluate model performance on NLST. All models are also evaluated on the Brock ILST 6\% (0.06) moderate risk threshold \citep{Lim2020-ilst}.'))

# Results Part 1

## ROC Tables

In [ ]:
nlst_demos_to_include = ['Age', 'Sex', 'Race', 'Education', 'Height', 'Weight', 'BMI']

In [ ]:
nlst_scan_res = pd.read_csv(f"{RESULTS_DIR}/auroc-nlst-5911.csv", index_col=0)
nlst_model_order = ['Venkadesh', 'Sybil year 1', 'PanCan2b']
nlst_scan_out = output.roc_results_pretty(nlst_scan_res, nlst_model_order, precision=TABLE_SCORE_PRECISION).drop(columns=['Malignant Scans'])
nlst_scan_out_demos = nlst_scan_out.xs('Demographics', level='Category').query('Attribute in @nlst_demos_to_include & Attribute not in @invalid_attributes_5911')
nlst_scan_out_demos

In [ ]:
print(nlst_scan_out_demos.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/resNLSTscanROC.tex",
    hrules=True, label='tab:resNLSTscanROC', environment='table*', column_format=f'll{"|ll"*len(nlst_model_order)}', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data',
    caption='AUROC (with 95\% confidence intervals) for models for demographic subgroups on NLST.'))

## ROC Plots

In [ ]:
_ = roc.plot_rocs_subgroups(nlst_5911, 'Sex', models={'Sybil (Year 1)': 'sybil_year1'}, two_subgroups=True, dataset_name="NLST", num_bootstraps=1000, imgpath=f"{FIG_DIR}/sybilROCgender5911.png")

In [ ]:
_ = roc.plot_rocs_subgroups(nlst_5911, 'BMI', models={'Sybil (Year 1)': 'sybil_year1'}, two_subgroups=True, dataset_name="NLST", num_bootstraps=1000, imgpath=f"{FIG_DIR}/sybilBMIroc5911.png")

In [ ]:
_ = roc.plot_rocs_subgroups(nlst_5911, 'Emphysema in Scan', models={'Sybil (Year 1)': 'sybil_year1'}, two_subgroups=True, dataset_name="NLST", num_bootstraps=1000, imgpath=f"{FIG_DIR}/sybilROCemphysema5911.png")

## Threshold Utilities

In [ ]:
threshold_nlst_5911 = pd.read_csv(f'{RESULTS_DIR}/threshold-perfs-nlst-5911.csv', index_col=0)
threshold_nlst_5911, nlst_5911_policies = output.threshold_stats_pretty(threshold_nlst_5911, nlst_5911_policies)

## Threshold Plots

In [ ]:
_ = threshold.plot_threshold_stats_subgroups(nlst_5911, "Race", dataset_name='NLST',
                                                policies=nlst_5911_policies[['90% Specificity']], models=output.NLST_5911_MODELCOLS, 
                                                stats=threshold_nlst_5911.query(f'attribute == "Race"'),
                                                plot_metrics=['Sensitivity'], diff=False, show_mb_count=False,
                                                imgpath=f"{FIG_DIR}/allTPRrace5911.png"
                                                )  

In [ ]:
_ = threshold.plot_threshold_stats_subgroups(nlst_5911, "BMI", dataset_name='NLST',
                                                policies=nlst_5911_policies[['90% Sensitivity']], models=output.NLST_5911_MODELCOLS, 
                                                stats=threshold_nlst_5911.query(f'attribute == "BMI"'),
                                                plot_metrics=['Specificity'], diff=False, show_mb_count=False,
                                                imgpath=f"{FIG_DIR}/allTNRbmi5911.png"
                                                )  

In [ ]:
_ = threshold.plot_threshold_stats_subgroups(nlst_5911, "Age", dataset_name='NLST',
                                                policies=nlst_5911_policies[['90% Sensitivity']], models=output.NLST_5911_MODELCOLS, 
                                                stats=threshold_nlst_5911.query(f'attribute == "Age"'),
                                                plot_metrics=['Specificity'], diff=False, show_mb_count=False,
                                                imgpath=f"{FIG_DIR}/allTNRage5911.png"
                                                )  

In [ ]:
_ = threshold.plot_threshold_stats_subgroups(nlst_5911, "Education", dataset_name='NLST',
                                                policies=nlst_5911_policies[['90% Sensitivity']], models=output.NLST_5911_MODELCOLS, 
                                                stats=threshold_nlst_5911.query(f'attribute == "Education"'),
                                                plot_metrics=['Specificity'], diff=False, show_mb_count=False,
                                                imgpath=f"{FIG_DIR}/allTNReducation5911.png"
                                                )  

## Threshold Tables

In [ ]:
pairwise_tpr_5911 = threshold.all_attribute_pairwise_comparisons(threshold_nlst_5911, metric='Sensitivity')
threshold_tpr = output.threshold_results_pretty(pairwise_tpr_5911, model_order=list(output.NLST_5911_MODELCOLS.keys()), metric='Sensitivity')
tpr_demo_5911 = threshold_tpr.xs("Demographics", level='Category').query('Attribute in @nlst_demos_to_include & Policy in @POLICIES_TO_USE & Attribute not in @invalid_attributes_5911')
tpr_demo_5911

In [ ]:
output.latex_rotate_multirow('90% Sensitivity')

In [ ]:
print(tpr_demo_5911.style
        .format_index(output.latex_rotate_multirow, escape='latex-math', axis=0)
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .to_latex(
    buf=f"{TAB_DIR}/resNLSTfullTPR.tex",
    hrules=True, label='tab:resNLSTfullTPR', environment='table*', column_format=f'lll{"|ll"*len(list(output.NLST_5911_MODELCOLS.keys()))}', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption='Sensitivity (with 95\% confidence intervals) for models for demographic subgroups on NLST. Single asterisk (*) = sensitivity of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

In [ ]:
pairwise_tnr_5911 = threshold.all_attribute_pairwise_comparisons(threshold_nlst_5911, metric='Specificity')
threshold_tnr = output.threshold_results_pretty(pairwise_tnr_5911, model_order=list(output.NLST_5911_MODELCOLS.keys()), metric='Specificity')
tnr_demo_5911 = threshold_tnr.xs("Demographics", level='Category').query('Attribute in @nlst_demos_to_include & Policy in @POLICIES_TO_USE & Attribute not in @invalid_attributes_5911')
tnr_demo_5911

In [ ]:
print(tnr_demo_5911.style
        .format_index(output.latex_rotate_multirow, escape='latex-math', axis=0)
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .to_latex(
    buf=f"{TAB_DIR}/resNLSTfullTNR.tex",
    hrules=True, label='tab:resNLSTfullTNR', environment='table*', column_format=f'lll{"|ll"*len(list(output.NLST_5911_MODELCOLS.keys()))}', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption='Specificity (with 95\% confidence intervals) for models for demographic subgroups on NLST. Single asterisk (*) = specificity of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

# Confounder Results

## Prevalence + ROC Tables

In [ ]:
nlst_5911_v2, _, _ = data.prep_nlst_preds(nlst_preds_nodule, nlst_democols_og, scanlevel=True, sybil=True, pretty=False, bin_num=True)

### Sex

In [ ]:
gender_roc = pd.read_csv(f'{RESULTS_DIR}/auroc-gender-by-factors-nlst-5911.csv', index_col=0)
gender_prev_roc, gender_topn_confounders = output.prevalence_plus_isolated_roc(
    nlst_5911_v2,
    'Gender',
    NLST_POSSIBLE_CONFOUNDERS,
    gender_roc,
    'Sybil (Year 1)',
    topn=TOP_N_CONFOUNDERS,
    result_prec=TABLE_SCORE_PRECISION,
)

print(gender_prev_roc.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/genderIsolationPlusROC.tex",
    hrules=True, label='tab:genderIsolationPlusROC', environment='sidewaystable*', column_format=f'll|rrrr|rrrr|l', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'The top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between sexes, and the AUROC scores of the Sybil (Year 1) model between men and women, isolating for them.'))

gender_prev_roc

In [ ]:
_ = roc.plot_rocs_isolate_confounder(
    nlst_5911,
    'Sex',
    'Work w/o Mask',
    models={'Sybil (Year 1)': 'sybil_year1'},
    dataset_name="NLST",
    num_bootstraps=1000,
    imgpath=f"{FIG_DIR}/sybilROCgender-WrkNoMask.png"
)

### Race

In [ ]:
cat_race_confounders, num_race_confounders = output.confounders_by_attribute(nlst_5911_v2, 'WhiteOrBlack', NLST_POSSIBLE_CONFOUNDERS, precision=1)
num_race_confounders

In [ ]:
num_race_confounders

In [ ]:
nlst_preds_nodule.groupby(['WhiteOrBlack'])[['NoduleType', 'label']].value_counts(dropna=True, normalize=True)

In [ ]:
cat_race_confounders.xs("Nodule")

In [ ]:
race_roc = pd.read_csv(f'{RESULTS_DIR}/auroc-race-by-factors-nlst-5911.csv', index_col=0)
race_prev_roc, race_topn_confounders = output.prevalence_plus_isolated_roc(
    nlst_5911_v2,
    'WhiteOrBlack',
    NLST_POSSIBLE_CONFOUNDERS,
    race_roc,
    'Venkadesh21',
    topn=TOP_N_CONFOUNDERS,
    result_prec=TABLE_SCORE_PRECISION,
)

print(race_prev_roc.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/raceIsolationPlusROC.tex",
    hrules=True, label='tab:raceIsolationPlusROC', environment='sidewaystable*', column_format=f'll|rrrr|rrrr|l', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'The top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between White and Black participants, and the AUROC scores of the Venkadesh21 model between these racial groups isolating for them.'))

race_prev_roc

In [ ]:
race_roc_all = output.roc_isolations_pretty(
        race_roc, 'WhiteOrBlack', 'Venkadesh21', precision=2
)

In [ ]:
race_roc_all.xs('Nodule')

In [ ]:
_ = roc.plot_rocs_isolate_confounder(
    nlst_5911,
    'Race',
    'Hypertension Diag.',
    models={'Venkadesh21': 'DL_cal'},
    dataset_name="NLST",
    num_bootstraps=1000,
    imgpath=f"{FIG_DIR}/venkROCrace-Hypertension.png"
)

### BMI

In [ ]:
bmi_roc = pd.read_csv(f'{RESULTS_DIR}/auroc-bmi-by-factors-nlst-5911.csv', index_col=0)
bmi_prev_roc, bmi_topn_confounders = output.prevalence_plus_isolated_roc(
    nlst_5911_v2,
    'Overweight',
    NLST_POSSIBLE_CONFOUNDERS,
    bmi_roc,
    'Sybil (Year 1)',
    topn=TOP_N_CONFOUNDERS,
    result_prec=TABLE_SCORE_PRECISION,
)

print(bmi_prev_roc.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/bmiIsolationPlusROC.tex",
    hrules=True, label='tab:bmiIsolationPlusROC', environment='sidewaystable*', column_format=f'll|rrrr|rrrr|l', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'The top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between high and low BMI participants, and the AUROC scores of the Sybil (Year 1) model between BMI groups isolating for them.'))

bmi_prev_roc

In [ ]:
_ = roc.plot_rocs_isolate_confounder(
    nlst_5911,
    'BMI',
    'Emphysema in Scan',
    models={'Sybil (Year 1)': 'sybil_year1'},
    dataset_name="NLST",
    num_bootstraps=1000,
    imgpath=f"{FIG_DIR}/sybilROCbmi-Emphysema.png"
)

In [ ]:
bmi_pancan_roc, bmi_topn_confounders = output.prevalence_plus_isolated_roc(
    nlst_5911_v2,
    'Overweight',
    NLST_POSSIBLE_CONFOUNDERS,
    bmi_roc,
    'PanCan2b',
    topn=TOP_N_CONFOUNDERS,
    result_prec=TABLE_SCORE_PRECISION,
)

print(bmi_pancan_roc.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/bmiIsolationPlusROCpancan.tex",
    hrules=True, label='tab:bmiIsolationPlusROCpancan', environment='sidewaystable*', column_format=f'll|rrrr|rrrr|l', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'The top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between high and low BMI participants, and the AUROC scores of the PanCan2b model between BMI groups isolating for them.'))

bmi_pancan_roc

### Education

In [ ]:
hs_or_more_roc = pd.read_csv(f'{RESULTS_DIR}/auroc-hs-or-more-by-factors-nlst-5911.csv', index_col=0)
hs_or_more_sybil_roc, hs_or_more_topn_confounders = output.prevalence_plus_isolated_roc(
    nlst_5911_v2,
    'HS-or-more',
    NLST_POSSIBLE_CONFOUNDERS,
    hs_or_more_roc,
    'Sybil (Year 1)',
    topn=TOP_N_CONFOUNDERS,
    result_prec=TABLE_SCORE_PRECISION,
)
hs_or_more_sybil_roc

In [ ]:
hs_or_more_venk21_roc, hs_or_more_topn_confounders = output.prevalence_plus_isolated_roc(
    nlst_5911_v2,
    'HS-or-more',
    NLST_POSSIBLE_CONFOUNDERS,
    hs_or_more_roc,
    'Venkadesh21',
    topn=TOP_N_CONFOUNDERS,
    result_prec=TABLE_SCORE_PRECISION,
)
hs_or_more_venk21_roc

In [ ]:
hs_or_more_pancan_roc, hs_or_more_topn_confounders = output.prevalence_plus_isolated_roc(
    nlst_5911_v2,
    'HS-or-more',
    NLST_POSSIBLE_CONFOUNDERS,
    hs_or_more_roc,
    'PanCan2b',
    topn=TOP_N_CONFOUNDERS,
    result_prec=TABLE_SCORE_PRECISION,
)

print(hs_or_more_pancan_roc.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/educationIsolationPlusROCpancan.tex",
    hrules=True, label='tab:educationIsolationPlusROCpancan', environment='sidewaystable*', column_format=f'll|rrrr|rrrr|l', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'The top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between have graduated high school or higher and those who have not, and the AUROC scores of the PanCan2b model between these subgroups isolating for them.'))


hs_or_more_pancan_roc

## Sensitivity and Specificity

### Sex

#### Sybil

In [ ]:
gender_pairwise_comps = None

In [ ]:
gender_isolations = pd.read_csv(f'{RESULTS_DIR}/threshold-gender-by-factors-nlst-5911.csv', index_col=0)
gender_tpr_tnr, gender_pairwise_comps = output.threshold_isolation_pairwise(
    gender_isolations, "Gender", "Sybil (Year 1)", 
    nlst_5911_policies, 
    topn_confs=gender_topn_confounders, 
    pairwise_comps=gender_pairwise_comps)
gender_tpr_tnr

In [ ]:
print(gender_tpr_tnr.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/genderTPRandTNRisolated.tex",
    hrules=True, label='tab:genderTPRandTNRisolated', environment='sidewaystable*', column_format=f'll|rrr|rrr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'Sensitivity and Specificity at specific thresholds (with 95\% confidence intervals) for the Sybil (Year 1) model between men and women, isolating for the top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between sexes. Single asterisk (*) = metric of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

#### PanCan2b

In [ ]:
pancan_gender_tpr_tnr, gender_pairwise_comps = output.threshold_isolation_pairwise(
    gender_isolations, "Gender", "PanCan2b", 
    nlst_5911_policies, 
    topn_confs=gender_topn_confounders, 
    pairwise_comps=gender_pairwise_comps)
pancan_gender_tpr_tnr

### Race

In [ ]:
race_pairwise_comps = None

In [ ]:
race_isolations = pd.read_csv(f'{RESULTS_DIR}/threshold-race-by-factors-nlst-5911.csv', index_col=0)
race_tpr_tnr, race_pairwise_comps = output.threshold_isolation_pairwise(
    race_isolations, "WhiteOrBlack", "Venkadesh21", 
    nlst_5911_policies, 
    topn_confs=race_topn_confounders, 
    pairwise_comps=race_pairwise_comps)
race_tpr_tnr

In [ ]:
print(race_tpr_tnr.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/raceTPRandTNRisolated.tex",
    hrules=True, label='tab:raceTPRandTNRisolated', environment='sidewaystable*', column_format=f'll|rrr|rrr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'Sensitivity and Specificity at specific thresholds (with 95\% confidence intervals) for the Venkadesh21 model between between White and Black participants, isolating for the top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between racial groups. Single asterisk (*) = metric of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

In [ ]:
all_race_tpr_tnr, _ = output.threshold_isolation_pairwise(
    race_isolations, "WhiteOrBlack", "Venkadesh21", 
    nlst_5911_policies, 
    topn_confs=None, 
    pairwise_comps=race_pairwise_comps)

In [ ]:
all_race_tpr_tnr[all_race_tpr_tnr.index.get_level_values(0).str.contains('Nodule')]

### BMI

In [ ]:
bmi_pairwise_comps = None

In [ ]:
bmi_isolations = pd.read_csv(f'{RESULTS_DIR}/threshold-bmi-by-factors-nlst-5911.csv', index_col=0)
bmi_tpr_tnr, bmi_pairwise_comps = output.threshold_isolation_pairwise(
    bmi_isolations, "Overweight", "Sybil (Year 1)", 
    nlst_5911_policies, 
    topn_confs=bmi_topn_confounders, 
    pairwise_comps=bmi_pairwise_comps)
bmi_tpr_tnr

In [ ]:
print(bmi_tpr_tnr.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/bmiTPRandTNRisolated.tex",
    hrules=True, label='tab:bmiTPRandTNRisolated', environment='sidewaystable*', column_format=f'll|rrr|rrr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'Sensitivity and Specificity at specific thresholds (with 95\% confidence intervals) for the Sybil (Year 1) model between between high and low BMI participants, isolating for the top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between subgroups. Single asterisk (*) = metric of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

In [ ]:
pancan_bmi_tpr_tnr, bmi_pairwise_comps = output.threshold_isolation_pairwise(
    bmi_isolations, "Overweight", "PanCan2b", 
    nlst_5911_policies, 
    topn_confs=bmi_topn_confounders, 
    pairwise_comps=bmi_pairwise_comps)

print(pancan_bmi_tpr_tnr.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/bmiTPRandTNRisolatedPanCan.tex",
    hrules=True, label='tab:bmiTPRandTNRisolatedPanCan', environment='sidewaystable*', column_format=f'll|rrr|rrr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'Sensitivity and Specificity at specific thresholds (with 95\% confidence intervals) for the PanCan2b model between between high and low BMI participants, isolating for the top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between subgroups. Single asterisk (*) = metric of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

pancan_bmi_tpr_tnr

In [ ]:
venk_bmi_tpr_tnr, bmi_pairwise_comps = output.threshold_isolation_pairwise(
    bmi_isolations, "Overweight", "Venkadesh21", 
    nlst_5911_policies, 
    topn_confs=bmi_topn_confounders, 
    pairwise_comps=bmi_pairwise_comps)

print(venk_bmi_tpr_tnr.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/bmiTPRandTNRisolatedVenk21.tex",
    hrules=True, label='tab:bmiTPRandTNRisolatedVenk21', environment='sidewaystable*', column_format=f'll|rrr|rrr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'Sensitivity and Specificity at specific thresholds (with 95\% confidence intervals) for the Venkadesh21 model between between high and low BMI participants, isolating for the top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between subgroups. Single asterisk (*) = metric of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))


venk_bmi_tpr_tnr

### Education

In [ ]:
hs_or_more_pairwise_comps = None

In [ ]:
hs_or_more_isolations = pd.read_csv(f'{RESULTS_DIR}/threshold-hs-or-more-by-factors-nlst-5911.csv', index_col=0)

In [ ]:
sybil_hs_or_more_tpr_tnr, hs_or_more_pairwise_comps = output.threshold_isolation_pairwise(
    hs_or_more_isolations, "HS-or-more", "Sybil (Year 1)", 
    nlst_5911_policies, 
    topn_confs=hs_or_more_topn_confounders, 
    pairwise_comps=hs_or_more_pairwise_comps)

print(sybil_hs_or_more_tpr_tnr.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/educationTPRandTNRisolatedSybil.tex",
    hrules=True, label='tab:educationTPRandTNRisolatedSybil', environment='sidewaystable*', column_format=f'll|rrr|rrr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'Sensitivity and Specificity at specific thresholds (with 95\% confidence intervals) for the Sybil Year 1 model between between participants who have graduated high school or higher and those who have not, isolating for the top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between subgroups. Single asterisk (*) = metric of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

sybil_hs_or_more_tpr_tnr

In [ ]:
venk21_hs_or_more_tpr_tnr, hs_or_more_pairwise_comps = output.threshold_isolation_pairwise(
    hs_or_more_isolations, "HS-or-more", "Venkadesh21", 
    nlst_5911_policies, 
    topn_confs=hs_or_more_topn_confounders, 
    pairwise_comps=hs_or_more_pairwise_comps)

print(venk21_hs_or_more_tpr_tnr.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/educationTPRandTNRisolatedVenk21.tex",
    hrules=True, label='tab:educationTPRandTNRisolatedVenk21', environment='sidewaystable*', column_format=f'll|rrr|rrr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'Sensitivity and Specificity at specific thresholds (with 95\% confidence intervals) for the Venkadesh21 model between between participants who have graduated high school or higher and those who have not, isolating for the top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between subgroups. Single asterisk (*) = metric of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

venk21_hs_or_more_tpr_tnr

In [ ]:
pancan_hs_or_more_tpr_tnr, hs_or_more_pairwise_comps = output.threshold_isolation_pairwise(
    hs_or_more_isolations, "HS-or-more", "PanCan2b", 
    nlst_5911_policies, 
    topn_confs=hs_or_more_topn_confounders, 
    pairwise_comps=hs_or_more_pairwise_comps)

print(pancan_hs_or_more_tpr_tnr.style
        .format(output.latex_replace_arrowbrackets, escape='latex-math', na_rep='')
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=1)
        .format_index(output.latex_replace_arrowbrackets, escape='latex-math', axis=0)
        .to_latex(
    buf=f"{TAB_DIR}/educationTPRandTNRisolatedPanCan.tex",
    hrules=True, label='tab:educationTPRandTNRisolatedPanCan', environment='sidewaystable*', column_format=f'll|rrr|rrr', position='h!',
    multirow_align='c', multicol_align='c',  position_float='centering', clines='skip-last;data', siunitx=True,
    caption=f'Sensitivity and Specificity at specific thresholds (with 95\% confidence intervals) for the PanCan2b model between between participants who have graduated high school or higher and those who have not, isolating for the top {TOP_N_CONFOUNDERS} characteristics with the largest prevalence difference between subgroups. Single asterisk (*) = metric of one subgroup is outside the CI of the other. Double asterisks (**) = CIs do not intersect.'))

pancan_hs_or_more_tpr_tnr